In [1]:
%matplotlib inline
# ---- settings ----
import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
sys.path.insert(0, '../'); sys.path.insert(0, '../samples')
import paf_loader
from os.path import join, isdir, isfile
import numpy as np
import cv2
from cselect import color as cs
from time import time
from reid import reid

from mvpose.data import umpm
from mvpose.algorithm.settings import get_settings, get_tracking_settings

root = join(Settings['data_root'], 'pak')
user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']
tmp = Settings['tmp']

_start = time()
X, Y, Calib = umpm.get(root, 'p2_free_1', user, pwd)

# --- Things that kinda have to be parametrized ---
#candidate_main_name = 'umpm_ms60_candidates_frame'
candidate_main_name = 'umpm_ms60_candidates_no_unary_frame'
dict_file = join(tmp, 'scores_s60_umpm_overfit.npy')
params = get_settings(ms_radius=60)
# -------------------------------------------------

_start = time()

#reid_model = reid.ReId(
#    url='http://188.138.127.15:81/models/model_heavy_umpm.h5',
#    name='model_heavy_umpm.h5')
reid_model = {}  # we can skip this ONLY if we already calc. reid scores!
tracking_params = get_tracking_settings(params, 
                                        low_spec_mode=True, 
                                        reid_model=reid_model,
                                        personreid_batchsize=2)
_end = time()
print("loading settings:", _end - _start)
def get_track(FRAMES):
    Ims = []
    Hms = []
    Pafs = []
    
    im_name = 'umpm_f' + '_'.join([str(s) for s in FRAMES]) + 'im.npy'
    hm_name = 'umpm_f' + '_'.join([str(s) for s in FRAMES]) + 'hm.npy'
    pf_name = 'umpm_f' + '_'.join([str(s) for s in FRAMES]) + 'pf.npy'
    im_name = join(tmp, im_name)
    hm_name = join(tmp, hm_name)
    pf_name = join(tmp, pf_name)
    
    if isfile(im_name) and isfile(hm_name) and isfile(pf_name):
        Ims = np.load(im_name)
        Hms = np.load(hm_name)
        Pafs = np.load(pf_name)
    else:
        for frame in FRAMES:
            print('load frame ', frame)
            Im = np.array([X[0][frame], X[1][frame], X[2][frame], X[3][frame]])
            Ims.append(Im)
            heatmaps, pafs = paf_loader.load_confidence_map_and_paf('umpm', 
                                                                Im, 
                                                                frame, 
                                                                dir=tmp)
            Hms.append(heatmaps)
            Pafs.append(pafs)

        Ims = np.array(Ims)
        Hms = np.array(Hms)
        Pafs = np.array(Pafs)
        np.save(im_name, Ims)
        np.save(hm_name, Hms)
        np.save(pf_name, Pafs)
        
    return Ims, Hms, Pafs

X[0].shape

/home/julian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


UMPM
loading settings: 0.0005092620849609375


(2437, 486, 644, 3)

# Step 1: pre-load pafs and heatmaps
(my laptop 'sucks' so I need to pre-load the pafs and heatmaps)

In [ ]:
# def load(FRAMES):
#     for frame in FRAMES:
#         print('handle frame ', frame)
#         Im = np.array([X[0][frame], X[1][frame], X[2][frame], X[3][frame]])
#         heatmaps, pafs = paf_loader.load_confidence_map_and_paf('umpm', 
#                                                             Im, 
#                                                             frame, 
#                                                             dir=tmp)
#         del heatmaps
#         del pafs
#         del Im

# def qq():
#     for i in range(45, 2500, 60):
#         FRAMES = list(range(i, i+60, 30))
#         print('frames:', FRAMES)
#         _start = time()
#         load(FRAMES)
#         _end = time()
#         print('\telapsed', _end - _start)

# qq()  # "memory-efficient" way of loading the data 

# Step 2: generate candidates

In [ ]:
# from mvpose import pose
# from mvpose.algorithm import graphcut

# def execute(FRAME):

#     Im = np.array([X[0][FRAME], X[1][FRAME], X[2][FRAME], X[3][FRAME]])

#     print('\nframe:', FRAME)
#     heatmaps, pafs = paf_loader.load_confidence_map_and_paf('umpm', 
#                                                             Im, 
#                                                             FRAME, 
#                                                             dir=tmp)
#     _start = time()
#     detections = pose.estimate_heuristic(
#         Calib, heatmaps, pafs, settings=params, 
#         debug=False, use_greedy=False)
#     _end = time()
#     print('total elapsed:', _end - _start)
    
#     # -- store to file --
#     candidate_name = candidate_main_name + str(FRAME) + ".npy"
#     candidate_name = join(tmp, candidate_name)
#     np.save(candidate_name, detections)
    
#     del heatmaps
#     del pafs
#     del detections

# def run_all():
#     # go to 2500
#     for i in range(2430, 2500, 60):
#         FRAMES = list(range(i, i+60, 30))
#         for f in FRAMES:
#             execute(f)

# run_all()



# Step 3: calculate re-id score

In [2]:
from mvpose.algorithm.track_graph_partitioning import get_bb, distance3d_humans, ValidImageCandidateExtractor
from mvpose.tracking import extend_calibration_to_frames

n_frames = 2437
graph_3d = {}
human_candidates = []
Im = []
for frame in range(0, n_frames, 30):
    if frame % 300 == 0:
        print('frame ', frame)
    candidate_name = candidate_main_name + str(frame) + ".npy"
    candidate_name = join(tmp, candidate_name)
    assert isfile(candidate_name)
    Humans = np.load(candidate_name)
    human_candidates.append(Humans)
    im = np.array([X[0][frame], X[1][frame], X[2][frame], X[3][frame]], 'uint8')
    Im.append(im)
Im = np.array(Im, 'uint8')

print(len(human_candidates))

Calibs = extend_calibration_to_frames([Calib], n_frames)

img_cand = ValidImageCandidateExtractor(
    Im, human_candidates, Calibs, tracking_params)


# ---------------------------------------------------

# n = len(img_cand.ImgsA)
# start = 0
# if isfile(dict_file):
#     start = np.load(dict_file).item()['next_frame']

# print('start:' + str(start) + ' until ' + str(n))
# for frame in range(start, n):
#     if isfile(dict_file):
#         storage = np.load(dict_file).item()
#     else:
#         storage = {}
    
#     if frame % 10 == 0:
#         print('frame %05d/' % (frame+1) + str(n))
#         _start = time()
#     A = img_cand.ImgsA[frame]
#     B = img_cand.ImgsB[frame]
#     score = np.squeeze(reid_model.predict(A, B))
#     #print('storage', storage)
#     #print('score', score)
#     storage[frame] = score
#     storage['next_frame'] = frame + 1
#     np.save(dict_file, storage)
#     if frame % 10 == 0:
#         _end = time()
#         print('\telapsed', _end - _start)
#     del storage
    

frame  0
frame  300
frame  600
frame  900
frame  1200
frame  1500
frame  1800
frame  2100
frame  2400
82


# Step 4: build graph

In [ ]:
# from mvpose.algorithm.track_graph_partitioning import generate_graph3d, GraphSolver
# from mvpose.algorithm.track_graph_partitioning import construct_query_graph
# from mvpose.tracking import extract_tracks

# storage = np.load(dict_file).item()

# scores = []
# pairs = img_cand.pairs
# n = len(img_cand.ImgsA)
# print(n)

# assert(len(pairs) == n)
# for frame in range(n):
#     scores.append(storage[frame])
# assert len(scores) == n


# graph_3d = generate_graph3d(scores, pairs)  # t1, pid1, t2, pid2

# n_frames_red = len(human_candidates)
# graph_partitioning = GraphSolver(graph_3d, 
#                                  n_frames_red,
#                                  tracking_params.T)


# G = construct_query_graph(graph_partitioning, n_frames_red)

# tracks = extract_tracks(human_candidates, G)

# Step 5: Evaluation

In [ ]:
# import numpy as np
# from pak.evaluation import MOTA

# from mvpose.tracking import extract_tracks

# tracks = extract_tracks(human_candidates, G)

# # right food: own dataset: 13, mscoco: 10
# # left foot: own dataset 10, mscoco: 13

# GT_jid = 13  # 0 ... 13
# HY_jid = 10  # 0 ... 17

# Gt = []
# Hy = []
# for loc_frame, frame in enumerate(range(0, n_frames, 30)):
#     # === ground truth ===
#     y = Y[frame]
#     pid1 = 0
#     p1 = y[GT_jid, 0:3]
#     pid2 = 1
#     p2 = y[GT_jid+14, 0:3]
#     Gt.append((loc_frame, pid1, *p1))
#     Gt.append((loc_frame, pid2, *p2))
    
#     # === hypothesis ===
#     for loc, human in enumerate(human_candidates[loc_frame]):
#         pt = human[HY_jid]
#         pid = tracks[loc_frame][loc]
#         if pt is not None:
#             Hy.append((loc_frame, pid, *pt))
    
    
# # print(Hy)
# # print('--')
# # print(Gt)
# Hy = np.array(Hy)
# Gt = np.array(Gt)

# threshold = 150
# # result is a scalar value in the range of [-infinity, 1)
# mota, info, debug = MOTA.evaluate(Gt, Hy, threshold, info=True, 
#                            debug_info=True)

# print('MOTA', mota)
# print('info', info)

## PCP score calculation

In [5]:
from mvpose.evaluation import pcp
import mvpose.data.kth_football2 as kth
import mvpose.data.skeleton_augmentation as ska
model_path = '../data/model_poseprediction.h5'
gen = ska.LimbGenerator(model_path, params.scale_to_mm)

def generate_pcp_score(frame):
    alpha = 0.5
    candidate_name = candidate_main_name + str(frame) + ".npy"
    candidate_name = join(tmp, candidate_name)
    assert isfile(candidate_name)
    candidates = np.load(candidate_name)
    candidates = gen.apply(candidates)
    Humans = kth.transform3d_from_mscoco(candidates)
    
    #Im, Y, Calib = epfl_campus.get(root, frame)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    Avg = []
    GTIDs = []
  
    
    a = Y[frame,0:14,0:3]
    b = Y[frame,14:,0:3]
    
    Y_ = kth.transform3d_from_umpm([a, b])

    for gtid, gt in enumerate(Y_):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        Avg.append(avg)
        GTIDs.append(gtid)
    
    del candidates
    #del Im
    #del Y
    #del Calib
    return L_Arms, U_Arms, L_Legs, U_Legs, Avg, GTIDs


PER_GTID = {}

n_frames = 2437
for frame in range(0, n_frames, 30):
    L_Arms, U_Arms, L_Legs, U_Legs, Avg, GTIDs =\
        generate_pcp_score(frame)
    
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs, avg in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs, Avg
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'avg': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['avg'].append(avg)
            PER_GTID[gtid]['frame'].append(frame)
            

In [6]:
for pid in PER_GTID.keys():
    print('pid=', pid)
    
    print('\tuarms:', np.mean(PER_GTID[pid]['uarms']))
    print('\tlarms:', np.mean(PER_GTID[pid]['larms']))
    print('\tulegs:', np.mean(PER_GTID[pid]['ulegs']))
    print('\tllegs:', np.mean(PER_GTID[pid]['llegs']))
    print('\tavg  :', np.mean(PER_GTID[pid]['avg']))

pid= 0
	uarms: 0.975609756097561
	larms: 0.9634146341463414
	ulegs: 1.0
	llegs: 1.0
	avg  : 0.9847560975609756
pid= 1
	uarms: 0.975609756097561
	larms: 0.9207317073170732
	ulegs: 0.9939024390243902
	llegs: 0.9939024390243902
	avg  : 0.9710365853658537
